# Task 4

## Setup

In [1]:
# --- Check Python and pip versions ---
!python --version
!pip install --upgrade pip

Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
# --- Install required libraries ---
!pip install torch
!pip install numpy pandas scikit-learn matplotlib seaborn
!pip install tqdm

In [3]:
# --- Setup & Imports ---
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter, defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.cuda.amp import GradScaler
from torch.nn.utils import clip_grad_norm_
from torch.cuda import is_available
from torch import cuda

from transformers import (
    AutoTokenizer, AutoModelForTokenClassification, AutoConfig,
    get_scheduler, DataCollatorForTokenClassification
)
from transformers import AutoTokenizer

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score, f1_score,
    precision_score, recall_score
)
from itertools import chain
from copy import deepcopy

from tqdm.auto import tqdm

### Colab Pro

In [4]:
# --- Check GPU availability ---
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [5]:
# --- Check RAM availability ---
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


### Paths setup


In [6]:
# --- Mount Google Drive (for Google Colab users) ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# --- Define Paths ---
laboratory = 'Laboratory4'

base_path = '/content/drive/MyDrive/'
project_path = base_path + f'Projects/{laboratory}/'
data_path = project_path + 'data/'
results_path = project_path + 'results/'

# Ensure directories exist
os.makedirs(project_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

print(f"Project path: {project_path}")
print(f"Data path: {data_path}")
print(f"Results path: {results_path}")

Project path: /content/drive/MyDrive/Projects/Laboratory4/
Data path: /content/drive/MyDrive/Projects/Laboratory4/data/
Results path: /content/drive/MyDrive/Projects/Laboratory4/results/


In [8]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [9]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

## Helper Functions

In [10]:
# ============================================================================
# GLOBAL CONFIGURATION: Plot Saving (Colab/Google Drive only)
# ============================================================================
SAVE_PLOTS = 1
# ============================================================================

import os
import matplotlib.pyplot as plt

BASE_DIR = results_path + 'Task4'
os.makedirs(BASE_DIR, exist_ok=True)

def save_figure_for_report(filename, dpi=300, bbox_inches='tight'):
    """
    Save the current matplotlib figure for use in the report.

    Args:
        filename: Name of the file (e.g., 'class_distribution.png')
        dpi: Resolution (default 300 for high quality)
        bbox_inches: Bounding box setting (default 'tight' to remove whitespace)
    """
    if not SAVE_PLOTS:
        return  # Skip saving if flag is disabled or filename missing

    filepath = os.path.join(BASE_DIR, filename)
    plt.savefig(filepath, dpi=dpi, bbox_inches=bbox_inches)
    print(f"Figure saved to: {filepath}")


In [11]:
def compute_metrics(full_predictions, full_labels):
    """Compute token-level classification metrics"""
    flat_predictions = list(chain(*full_predictions))
    flat_labels = list(chain(*full_labels))

    token_accuracy = accuracy_score(flat_labels, flat_predictions)
    token_precision = precision_score(flat_labels, flat_predictions, average='macro', zero_division=0)
    token_recall = recall_score(flat_labels, flat_predictions, average='macro', zero_division=0)
    token_f1 = f1_score(flat_labels, flat_predictions, average='macro', zero_division=0)

    metrics = {
        "token_accuracy": token_accuracy,
        "token_precision": token_precision,
        "token_recall": token_recall,
        "token_f1": token_f1,
    }
    return metrics

In [12]:
def postprocess(predictions, labels):
    """Convert predictions and labels to original label format"""
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_predictions, true_labels

In [13]:
def align_labels_with_tokens(labels, word_ids):
    """Align word-level labels to token-level labels"""
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            new_labels.append(label)
    return new_labels

In [14]:
def tokenize_and_align_labels_unixcoder(samples, tokenizer):
    """Tokenize and align labels for UniXcoder"""
    split_sentences = [s.split(" ") for s in samples["session"]]

    tokenized = tokenizer(
        split_sentences,
        truncation=True,
        is_split_into_words=True
    )

    all_labels = samples["label_id"]
    aligned_all = []

    for i, labels in enumerate(all_labels):
        word_ids = tokenized.word_ids(i)

        aligned = []
        prev_word = None

        for wid in word_ids:
            if wid is None:
                aligned.append(-100)
            else:
                if wid != prev_word:
                    aligned.append(labels[wid])
                    prev_word = wid
                else:
                    aligned.append(-100)
        aligned_all.append(aligned)

    tokenized["labels"] = aligned_all
    return tokenized


In [15]:
def training_loop(model, optimizer, lr_scheduler, train_loader, val_loader, device, num_epochs):
    scaler = torch.amp.GradScaler()
    best_val_loss = np.inf
    best_weights = deepcopy(model.state_dict())

    # Calculate steps dynamically based on the passed loader
    num_training_steps = num_epochs * len(train_loader)
    progress_bar = tqdm(range(num_training_steps))

    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            # Move batch to device
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            optimizer.zero_grad(set_to_none=True)

            # Automatic Mixed Precision
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(**batch)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()

            train_loss += loss.item()
            progress_bar.update(1)

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Evaluation
        model.eval()
        val_loss = 0
        predictions_list, labels_list = [], []

        for batch in val_loader:
            batch = {key: value.to(device, non_blocking=True) for key, value in batch.items()}
            with torch.no_grad():
                with torch.amp.autocast(device_type='cuda'):  # ← FIXED HERE
                    outputs = model(**batch)

            val_loss += outputs.loss.item()
            predictions = outputs.logits.argmax(dim=-1)
            labels = batch["labels"]

            true_predictions, true_labels = postprocess(predictions, labels)
            predictions_list += true_predictions
            labels_list += true_labels

        # Compute validation metrics
        val_metrics = compute_metrics(predictions_list, labels_list)
        val_accuracy = val_metrics["token_accuracy"]

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        # Optional: Print progress
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss <= best_val_loss:
            best_epoch = epoch
            best_val_loss = avg_val_loss
            best_weights = deepcopy(model.state_dict())

    # Load the best weights found during this specific training run
    model.load_state_dict(best_weights)
    return model, best_epoch, best_val_loss, train_losses, val_losses

In [16]:
def evaluate_model(model, dataloader, device):
    """Evaluate model on a dataset"""
    model.eval()
    full_predictions, full_labels = [], []
    for batch in dataloader:
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        true_predictions, true_labels = postprocess(predictions, labels)
        full_predictions += true_predictions
        full_labels += true_labels

    test_metrics = compute_metrics(full_predictions, full_labels)
    return full_predictions, full_labels, test_metrics

## Re-Train Best Model from Task 3

### Dataset Loading and preparation

In [17]:
# Load training and test data
train_df = pd.read_json(f"{data_path}train.json")
test_df = pd.read_json(f"{data_path}test.json")

print(f"Training dataset: {train_df.shape[0]} sessions")
print(f"Test dataset: {test_df.shape[0]} sessions")

Training dataset: 251 sessions
Test dataset: 108 sessions


In [18]:
# Split training data into train and validation
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
print(f"Split training: {train_df.shape[0]} train, {val_df.shape[0]} validation")

Split training: 200 train, 51 validation


In [19]:
# Create label mappings
unique_labels = list(train_df.label.explode().unique())
print(f"Unique labels: {unique_labels}")

id2label = {it: label for it, label in enumerate(unique_labels)}
label2id = {label: it for it, label in enumerate(unique_labels)}

print(f"Label mappings: {label2id}")

Unique labels: ['Execution', 'Discovery', 'Not Malicious Yet', 'Persistence', 'Other', 'Defense Evasion', 'Impact']
Label mappings: {'Execution': 0, 'Discovery': 1, 'Not Malicious Yet': 2, 'Persistence': 3, 'Other': 4, 'Defense Evasion': 5, 'Impact': 6}


In [20]:
# Create Hugging Face datasets
full_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "valid": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True)),
})

In [21]:
# Convert labels to IDs
def convert_labels_to_ids(sample):
    sample['label_id'] = [label2id[el] for el in sample["label"]]
    return sample

encoded_dataset = full_ds.map(convert_labels_to_ids)
print(f"Encoded dataset: {encoded_dataset}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

Encoded dataset: DatasetDict({
    train: Dataset({
        features: ['session', 'label', 'label_id'],
        num_rows: 200
    })
    valid: Dataset({
        features: ['session', 'label', 'label_id'],
        num_rows: 51
    })
    test: Dataset({
        features: ['session', 'label', 'label_id'],
        num_rows: 108
    })
})


### Tokenization

In [22]:
# Create tokenizer and model checkpoint
unixcoder_model_checkpoint = "microsoft/unixcoder-base"
unixcoder_tokenizer = AutoTokenizer.from_pretrained(
    unixcoder_model_checkpoint,
    add_prefix_space=True,
    use_fast=True,
    model_max_length=512
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [23]:
# Tokenize datasets
original_columns = encoded_dataset["train"].column_names
tokenized_datasets = encoded_dataset.map(
    lambda x: tokenize_and_align_labels_unixcoder(x, unixcoder_tokenizer),
    batched=True,
    remove_columns=original_columns,
)

print(f"Tokenized dataset: {tokenized_datasets}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

Tokenized dataset: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 51
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 108
    })
})


In [24]:
# Create data collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=unixcoder_tokenizer,
    return_tensors="pt"
)

In [25]:
# Create DataLoaders
tokenized_datasets.set_format("torch")

BATCH_SIZE = 32
NUM_WORKERS = 2

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

eval_dataloader = DataLoader(
    tokenized_datasets["valid"],
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_dataloader = DataLoader(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print("DataLoaders created successfully")

DataLoaders created successfully


### Train the model

In [26]:
print("\n" + "="*80)
print("SECTION 3: TRAIN BEST MODEL FROM TASK 3")
print("UNIXCODER FULL FINE-TUNE WITH LR=1e-05")
print("="*80)

N_TRAIN_EPOCHS = 40
BEST_LR = 1e-5

print(f"\nTraining parameters:")
print(f"  - Model: UniXcoder (microsoft/unixcoder-base)")
print(f"  - Learning Rate: {BEST_LR}")
print(f"  - Epochs: {N_TRAIN_EPOCHS}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Device: {device}")


SECTION 3: TRAIN BEST MODEL FROM TASK 3
UNIXCODER FULL FINE-TUNE WITH LR=1e-05

Training parameters:
  - Model: UniXcoder (microsoft/unixcoder-base)
  - Learning Rate: 1e-05
  - Epochs: 40
  - Batch Size: 32
  - Device: cpu


In [27]:
# Initialize model
best_model = AutoModelForTokenClassification.from_pretrained(
    unixcoder_model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
).to(device)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at microsoft/unixcoder-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Setup optimizer and scheduler
optimizer = optim.AdamW(best_model.parameters(), lr=BEST_LR)

num_training_steps = N_TRAIN_EPOCHS * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# Train
best_model, best_epoch, best_val_loss, train_losses, val_losses = training_loop(
    best_model, optimizer, lr_scheduler,
    train_dataloader, eval_dataloader, device, N_TRAIN_EPOCHS
)

print("\n" + "="*80)
print(f"Training complete! Best epoch: {best_epoch + 1}, Best val loss: {best_val_loss:.4f}")
print("="*80)

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

  0%|          | 0/280 [00:00<?, ?it/s]

In [ ]:
# Evaluate on test set
print("\nEvaluating on test set...")
test_preds, test_labels, test_metrics = evaluate_model(best_model, test_dataloader, device)

print(f"\nTest Set Performance:")
print(f"  - Token Accuracy: {test_metrics['token_accuracy']:.4f}")
print(f"  - Macro F1-Score: {test_metrics['token_f1']:.4f}")
print(f"  - Macro Precision: {test_metrics['token_precision']:.4f}")
print(f"  - Macro Recall: {test_metrics['token_recall']:.4f}")

In [ ]:
# # Save best model for later use in inference
# model_save_path = os.path.join(results_path, "best_unixcoder_model_task3")
# os.makedirs(model_save_path, exist_ok=True)
# best_model.save_pretrained(model_save_path)
# unixcoder_tokenizer.save_pretrained(model_save_path)

# print(f"\nBest model saved to: {model_save_path}")

## Inference Datasets

In [ ]:
# Load cyberlab.csv
cyberlab_df = pd.read_csv(f"{data_path}cyberlab.csv")

print(f"Cyberlab dataset: {len(cyberlab_df)} sessions")
print(f"Columns: {list(cyberlab_df.columns)}")
print(f"Data types:\n{cyberlab_df.dtypes}")
print(f"\nFirst few rows:")
print(cyberlab_df.head(3))

# Convert timestamps_statements to datetime
cyberlab_df['timestamps_statements'] = pd.to_datetime(cyberlab_df['timestamps_statements'])
cyberlab_df['date'] = cyberlab_df['timestamps_statements'].dt.date

print(f"\nDate range: {cyberlab_df['date'].min()} to {cyberlab_df['date'].max()}")

## Inference Functions

## Batch Inference on Cyberlab Dataset